# Hyperparameter Tuning using the Scikit Learn Wrapper.
Tuning hyperparameters of a Keras model using the Scikit Learn Wrapper<br>

Based on Gilbert Tanner Tutorial

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
np.random.seed(0)

iris = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', names=['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'label'])

In [3]:
le = LabelEncoder()
iris['label'] = le.fit_transform(iris['label'])
X = np.array(iris.drop(['label'], axis=1))
y = np.array(iris['label'])
y = to_categorical(y, num_classes=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


# Use a Keras Model in Scikit Learn

need to use the KerasClassifier or KerasRegressor

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

def c_model():
    model = Sequential()
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=c_model, epochs=50, batch_size=32)

In [5]:
history=model.fit(X_train, y_train)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
120/120 [==============================] - 1s 5ms/step - loss: 1.3464 - acc: 0.3250
Epoch 2/50
120/120 [==============================] - 0s 112us/step - loss: 1.2079 - acc: 0.3250
Epoch 3/50
120/120 [==============================] - 0s 102us/step - loss: 1.1073 - acc: 0.3250
Epoch 4/50
120/120 [==============================] - 0s 69us/step - loss: 1.0347 - acc: 0.4750
Epoch 5/50
120/120 [==============================] - 0s 74us/step - loss: 0.9978 - acc: 0.6917
Epoch 6/50
120/120 [==============================] - 0s 102us/step - loss: 0.9757 - acc: 0.6750
Epoch 7/50
120/120 [==============================] - 0s 104us/step - loss: 0.9623 - acc: 0.6167
Epoch 8/50
120/120 [==============================] - 0s 125us/step - loss: 0.9488 - acc: 0.5917
Epoch 9/50
120/120 [==============================] - 0s 93us/step - loss: 0.9341 - acc: 0.6167
Epoch 10/50


# Parameter Tuning using GridSearchCV

In [6]:
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=c_model)

batch_sizes = [10, 20, 50, 100]
epochs = [5, 10, 50]
parameters = {'batch_size': batch_sizes, 'epochs': epochs}
clf = GridSearchCV(model, parameters)
clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/5
80/80 [==============================] - 0s 3ms/step - loss: 1.9466 - acc: 0.3875
Epoch 2/5
80/80 [==============================] - 0s 243us/step - loss: 1.3835 - acc: 0.3875
Epoch 3/5
80/80 [==============================] - 0s 220us/step - loss: 1.1589 - acc: 0.5625
Epoch 4/5
80/80 [==============================] - 0s 181us/step - loss: 1.0511 - acc: 0.6625
Epoch 5/5
40/40 [==============================] - 0s 2ms/step
Epoch 1/5
80/80 [==============================] - 0s 4ms/step - loss: 1.7844 - acc: 0.2750
Epoch 2/5
80/80 [==============================] - 0s 184us/step - loss: 1.4095 - acc: 0.2750
Epoch 3/5
80/80 [==============================] - 0s 205us/step - loss: 1.1819 - acc: 0.4125
Epoch 4/5
80/80 [==============================] - 0s 239us/step - loss: 1.0421 - acc: 0.6375
Epoch 5/5
40/40 [==============================] - 0s 2ms/step
Epoch 1/5
80/80 [==============================] - 0s 5ms/step - loss: 1.6176 - acc: 0.3125
Epoch 2/5
80/80 [=================

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f212cad3828>,
             iid='warn', n_jobs=None,
             param_grid={'batch_size': [10, 20, 50, 100],
                         'epochs': [5, 10, 50]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [7]:
print(clf.best_score_, clf.best_params_)
means = clf.cv_results_['mean_test_score']
parameters = clf.cv_results_['params']
for mean, parammeter in zip(means, parameters):
    print(mean, parammeter)

0.9583333283662796 {'batch_size': 10, 'epochs': 50}
0.6666666716337204 {'batch_size': 10, 'epochs': 5}
0.6250000099341074 {'batch_size': 10, 'epochs': 10}
0.9583333283662796 {'batch_size': 10, 'epochs': 50}
0.5916666636864344 {'batch_size': 20, 'epochs': 5}
0.5333333412806193 {'batch_size': 20, 'epochs': 10}
0.9000000059604645 {'batch_size': 20, 'epochs': 50}
0.38333333532015484 {'batch_size': 50, 'epochs': 5}
0.5 {'batch_size': 50, 'epochs': 10}
0.6499999761581421 {'batch_size': 50, 'epochs': 50}
0.30833334227403003 {'batch_size': 100, 'epochs': 5}
0.3750000049670537 {'batch_size': 100, 'epochs': 10}
0.6083333293596903 {'batch_size': 100, 'epochs': 50}


### Tuning optimizer:

In [8]:
def c_model(optimizer):
    model = Sequential()
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=c_model, epochs=50, batch_size=32)
parameters = {'optimizer':['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']}
clf = GridSearchCV(model, parameters)
clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/50
80/80 [==============================] - 2s 30ms/step - loss: 1.5022 - acc: 0.3375
Epoch 2/50
80/80 [==============================] - 0s 130us/step - loss: 1.0859 - acc: 0.3375
Epoch 3/50
80/80 [==============================] - 0s 108us/step - loss: 0.9819 - acc: 0.4000
Epoch 4/50
80/80 [==============================] - 0s 116us/step - loss: 0.9278 - acc: 0.6250
Epoch 5/50
80/80 [==============================] - 0s 128us/step - loss: 0.8897 - acc: 0.7250
Epoch 6/50
80/80 [==============================] - 0s 136us/step - loss: 0.8542 - acc: 0.7250
Epoch 7/50
80/80 [==============================] - 0s 141us/step - loss: 0.8247 - acc: 0.7250
Epoch 8/50
80/80 [==============================] - 0s 134us/step - loss: 0.8026 - acc: 0.7250
Epoch 9/50
80/80 [==============================] - 0s 130us/step - loss: 0.7802 - acc: 0.7250
Epoch 10/50
80/80 [==============================] - 0s 133us/step - loss: 0.7610 - acc: 0.7250
Epoch 11/50
80/80 [==============================]

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f211c0162b0>,
             iid='warn', n_jobs=None,
             param_grid={'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta',
                                       'Adam', 'Adamax', 'Nadam']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [9]:
print(clf.best_score_, clf.best_params_)
means = clf.cv_results_['mean_test_score']
parameters = clf.cv_results_['params']
for mean, parammeter in zip(means, parameters):
    print(mean, parammeter)

0.95 {'optimizer': 'Adagrad'}
0.7583333333333333 {'optimizer': 'SGD'}
0.875 {'optimizer': 'RMSprop'}
0.95 {'optimizer': 'Adagrad'}
0.8166666666666667 {'optimizer': 'Adadelta'}
0.8916666666666667 {'optimizer': 'Adam'}
0.8083333333333333 {'optimizer': 'Adamax'}
0.95 {'optimizer': 'Nadam'}


### Tuning activation function:

In [10]:
def c_model(activation):
    model = Sequential()
    model.add(Dense(32, activation=activation))
    model.add(Dense(16, activation=activation))
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=c_model, epochs=50, batch_size=32)
parameters = {'activation':['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']}
clf = GridSearchCV(model, parameters)
clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/50
80/80 [==============================] - 4s 46ms/step - loss: 1.0912 - acc: 0.3875
Epoch 2/50
80/80 [==============================] - 0s 144us/step - loss: 1.0908 - acc: 0.3875
Epoch 3/50
80/80 [==============================] - 0s 142us/step - loss: 1.0903 - acc: 0.3875
Epoch 4/50
80/80 [==============================] - 0s 124us/step - loss: 1.0900 - acc: 0.3875
Epoch 5/50
80/80 [==============================] - 0s 136us/step - loss: 1.0897 - acc: 0.3875
Epoch 6/50
80/80 [==============================] - 0s 130us/step - loss: 1.0893 - acc: 0.3875
Epoch 7/50
80/80 [==============================] - 0s 157us/step - loss: 1.0889 - acc: 0.3875
Epoch 8/50
80/80 [==============================] - 0s 146us/step - loss: 1.0886 - acc: 0.3875
Epoch 9/50
80/80 [==============================] - 0s 126us/step - loss: 1.0881 - acc: 0.3875
Epoch 10/50
80/80 [==============================] - 0s 143us/step - loss: 1.0878 - acc: 0.3875
Epoch 11/50
80/80 [==============================]

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7f2115db99e8>,
             iid='warn', n_jobs=None,
             param_grid={'activation': ['softmax', 'softplus', 'softsign',
                                        'relu', 'tanh', 'sigmoid',
                                        'hard_sigmoid', 'linear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [11]:
print(clf.best_score_, clf.best_params_)
means = clf.cv_results_['mean_test_score']
parameters = clf.cv_results_['params']
for mean, parammeter in zip(means, parameters):
    print(mean, parammeter)

0.95 {'activation': 'tanh'}
0.23333333333333334 {'activation': 'softmax'}
0.7916666666666666 {'activation': 'softplus'}
0.9166666666666666 {'activation': 'softsign'}
0.75 {'activation': 'relu'}
0.95 {'activation': 'tanh'}
0.7 {'activation': 'sigmoid'}
0.6333333333333333 {'activation': 'hard_sigmoid'}
0.9333333333333333 {'activation': 'linear'}
